In [1]:
import wmi
import ctypes
from ctypes import wintypes
import wmi
from monitorcontrol import get_monitors


In [2]:
def get_screen_count():
    """
    获取当前连接的屏幕数量。
    """
    user32 = ctypes.windll.user32
    return user32.GetSystemMetrics(80)  # SM_CMONITORS

def get_brightness(screen_index=0):
    """
    获取指定屏幕的亮度。
    WMI 接口默认按顺序返回所有屏幕的亮度信息。
    """
    c = wmi.WMI(namespace='root\\WMI')
    brightness_methods = c.WmiMonitorBrightness()
    if screen_index < len(brightness_methods):
        return brightness_methods[screen_index].CurrentBrightness
    return None

def set_brightness(brightness, screen_index=0):
    """
    设置指定屏幕的亮度。
    """
    c = wmi.WMI(namespace='root\\WMI')
    brightness_methods = c.WmiMonitorBrightnessMethods()
    if screen_index < len(brightness_methods):
        brightness_methods[screen_index].WmiSetBrightness(brightness, 0)  # 延迟时间为0毫秒
        print(f"屏幕 {screen_index + 1} 的亮度已设置为 {brightness}%")
    else:
        print(f"屏幕 {screen_index + 1} 不存在")


In [ ]:
from pynput.mouse import Listener
import time

class MouseTracker:
    def __init__(self, detection_interval=1, max_cnt=10):
        self.last_position = None  # 上一次鼠标位置
        self.current_position = None  # 当前鼠标位置
        self.is_moving = False  # 鼠标是否在移动
        self.state_counter = 0  # 状态计数器
        self.MAX_Cnt = max_cnt  # 最大静止计数
        self.last_brightness = None  # 保存的屏幕亮度
        self.detection_interval = detection_interval  # 检测间隔时间（秒）

    def _on_move(self, x, y):
        """
        当鼠标移动时更新当前位置
        """
        self.current_position = (x, y)

    def _detect_motion(self):
        """
        定时检测鼠标是否静止或移动，并更新状态计数器
        """
        while True:
            if self.current_position != self.last_position:
                # 状态改变：鼠标开始移动
                if not self.is_moving:
                    print("鼠标开始移动")
                    if self.state_counter == self.MAX_Cnt:
                        self._restore_brightness()
                self.is_moving = True
                self.state_counter = 0  # 状态重置
                self.last_position = self.current_position
            else:
                # 状态未改变
                if self.is_moving:
                    print("鼠标静止")
                self.is_moving = False
                if self.state_counter < self.MAX_Cnt:
                    self.state_counter += 1

                if self.state_counter == self.MAX_Cnt:
                    self._dim_screen()

            print(f"当前状态计数器: {self.state_counter}")
            time.sleep(self.detection_interval)

    def _dim_screen(self):
        """
        将屏幕亮度设置为 0，并保存当前亮度
        """
        
        if self.last_brightness is None:  # 避免重复操作
            self.last_brightness = self.get_brightness()
            self.set_brightness(0)
            print(f"屏幕已变暗，保存的亮度为: {self.last_brightness}")

    def _restore_brightness(self):
        """
        恢复屏幕亮度到静止前的亮度
        """
        if self.last_brightness is not None:
            self.set_brightness(self.last_brightness)
            print(f"屏幕亮度恢复至: {self.last_brightness}")
            self.last_brightness = None

    def get_brightness(self):
        """
        获取当前屏幕亮度（模拟实现，需替换为实际函数）
        """
        return get_brightness()  # 示例值，替换为实际获取亮度的代码

    def set_brightness(self, value):
        """
        设置屏幕亮度（模拟实现，需替换为实际函数）
        """
        print(f"设置屏幕亮度为: {value}%")  # 替换为实际设置亮度的代码
        set_brightness(value)

    def start(self):
        """
        开始检测鼠标位置和状态
        """
        print("开始检测鼠标状态...")
        listener = Listener(on_move=self._on_move)
        listener.start()

        try:
            self._detect_motion()
        except KeyboardInterrupt:
            print("检测已停止")
            listener.stop()

# 初始化并启动鼠标检测
tracker = MouseTracker(detection_interval = 1, max_cnt = 10)
tracker.start()


开始检测鼠标状态...
当前状态计数器: 1
鼠标开始移动
当前状态计数器: 0
当前状态计数器: 0
鼠标静止
当前状态计数器: 1
当前状态计数器: 2
当前状态计数器: 3
当前状态计数器: 4
当前状态计数器: 5
当前状态计数器: 6
当前状态计数器: 7
当前状态计数器: 8
当前状态计数器: 9
设置屏幕亮度为: 0%
屏幕 1 的亮度已设置为 0%
屏幕已变暗，保存的亮度为: 100
当前状态计数器: 10
当前状态计数器: 10
当前状态计数器: 10
当前状态计数器: 10
当前状态计数器: 10
当前状态计数器: 10
当前状态计数器: 10
当前状态计数器: 10
当前状态计数器: 10
当前状态计数器: 10
当前状态计数器: 10
当前状态计数器: 10
当前状态计数器: 10
当前状态计数器: 10
当前状态计数器: 10
当前状态计数器: 10
当前状态计数器: 10
当前状态计数器: 10
当前状态计数器: 10
当前状态计数器: 10
当前状态计数器: 10
鼠标开始移动
设置屏幕亮度为: 100%
屏幕 1 的亮度已设置为 100%
屏幕亮度恢复至: 100
当前状态计数器: 0
当前状态计数器: 0
当前状态计数器: 0
当前状态计数器: 0
当前状态计数器: 0
鼠标静止
当前状态计数器: 1
当前状态计数器: 2
当前状态计数器: 3
鼠标开始移动
当前状态计数器: 0
鼠标静止
当前状态计数器: 1
当前状态计数器: 2
当前状态计数器: 3
当前状态计数器: 4
当前状态计数器: 5
当前状态计数器: 6
当前状态计数器: 7
当前状态计数器: 8
当前状态计数器: 9
设置屏幕亮度为: 0%
屏幕 1 的亮度已设置为 0%
屏幕已变暗，保存的亮度为: 100
当前状态计数器: 10
当前状态计数器: 10
当前状态计数器: 10
当前状态计数器: 10
当前状态计数器: 10
当前状态计数器: 10
当前状态计数器: 10
当前状态计数器: 10
当前状态计数器: 10
当前状态计数器: 10
鼠标开始移动
设置屏幕亮度为: 100%
屏幕 1 的亮度已设置为 100%
屏幕亮度恢复至: 100
当前状态计数器: 0
当前状态计数器: 0
当前状态计数器: 0
当前状态计数器: 0
鼠标静止
当前状态计数器: 1
鼠

In [12]:
get_brightness()

set_brightness(60)


屏幕 1 的亮度已设置为 60%


In [24]:

def adjust_brightness_internal(brightness, screen_index=0):
    """
    调整内置屏幕亮度。
    """
    c = wmi.WMI(namespace='root\\WMI')
    brightness_methods = c.WmiMonitorBrightnessMethods()
    if screen_index < len(brightness_methods):
        brightness_methods[screen_index].WmiSetBrightness(brightness, 0)
        print(f"内置屏幕 {screen_index + 1} 的亮度已设置为 {brightness}%")
    else:
        print(f"内置屏幕 {screen_index + 1} 不存在或不支持亮度调节")

def adjust_brightness_external(brightness):
    """
    调整外接显示器亮度。
    """
    monitors = get_monitors()
    for i, monitor in enumerate(monitors):
        print(f"设置外接屏幕 {i + 1} 的亮度为 {brightness}%")
        with monitor:
            monitor.set_luminance(brightness)


In [4]:
# 检测屏幕数量
screen_count = get_screen_count()
print(f"检测到 {screen_count} 个屏幕")


检测到 1 个屏幕


In [25]:
target_brightness = 0


# 调整内置屏幕亮度
adjust_brightness_internal(target_brightness)


内置屏幕 1 的亮度已设置为 0%
